In [10]:
import rasterio
from affine import Affine

import climatezones as cz 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
points = cz.get_points()
points = points[~points.long.isnull()]
points

In [2]:
gelu = rasterio.open('dbs/geospatial/globalelu/World_ELU_2015.tif')
gelus = rasterio.open('dbs/geospatial/globalelus_query/World_Ecological_2015.tif')

In [43]:
gelu.height

67020

In [4]:
transformer = rasterio.transform.AffineTransformer(gelu.transform)


In [6]:
transformer.rowcol(-130.5, 75)

(4139, 23760)

In [13]:
transformer.rowcol?

Signature:
transformer.rowcol(
    xs,
    ys,
    zs=None,
    op=<built-in function floor>,
    precision=None,
)
Docstring:
Get rows and cols coordinates given geographic coordinates.

Parameters
----------
xs, ys : float or list of float
    Geographic coordinates
zs : float or list of float, optional
    Height associated with coordinates. Primarily used for RPC based 
    coordinate transformations. Ignored for affine based 
    transformations. Default: 0.
op : function, optional (default: math.floor)
    Function to convert fractional pixels to whole numbers (floor,
    ceiling, round)
precision : int, optional (default: None)
    This parameter is unused, deprecated in rasterio 1.3.0, and
    will be removed in version 2.0.0.

Raises
------
ValueError
    If input coordinates are not all equal length

Returns
-------
tuple of float or list of float.
File:      ~/.pyenv/versions/miniforge3/envs/image-classifier/lib/python3.9/site-packages/rasterio/transform.py
Type:      method

In [34]:
band1 = gelu.read(1)

In [104]:
band1[13413, 123360]

460

In [100]:
points['x'], points['y'] = transformer.rowcol(points.long, points.lat)


In [102]:
fpoints = points[points.x <= gelu.height]
fpoints['elu'] = band1[fpoints.x, fpoints.y]

/Users/weishuhn/.pyenv/versions/miniforge3/envs/image-classifier/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [103]:
fpoints.groupby(by="elu").count()

,gbifid,lat,long,geometry,x,y
elu,,,,,,
0,1105715,1105715,1105715,1105715,1105715,1105715
1,5114,5114,5114,5114,5114,5114
2,1355,1355,1355,1355,1355,1355
3,198,198,198,198,198,198
4,203,203,203,203,203,203
...,...,...,...,...,...,...
3573,78,78,78,78,78,78
3600,9,9,9,9,9,9
3610,7,7,7,7,7,7


In [95]:
fpoints

,gbifid,lat,long,geometry,x,y,elu
pointid,,,,,,,
0,1431462551,56.218360,16.416320,POINT (56.218 16.416),13155,94279,0
1,1438086806,58.448580,13.701640,POINT (58.449 13.702),12084,92976,372
2,1669971044,59.551380,12.236730,POINT (59.551 12.237),11555,92273,301
3,2270949504,59.845890,17.567560,POINT (59.846 17.568),11413,94832,451
4,2271059142,59.262680,15.548110,POINT (59.263 15.548),11693,93863,451
...,...,...,...,...,...,...,...
13902393,2828511327,-31.895833,116.557222,POINT (-31.896 116.557),55449,142347,2072
13902394,2830380118,-37.491667,148.702500,POINT (-37.492 148.702),58135,157777,1750
13902395,2830264695,-32.316667,152.416667,POINT (-32.317 152.417),55651,159560,1372


In [101]:
points[points.x > 66020]

,gbifid,lat,long,geometry,x,y
pointid,,,,,,
467253,2574153427,-55.946671,-67.275052,POINT (-55.947 -67.275),66994,54107
470858,3118391405,-54.844341,-68.573493,POINT (-54.844 -68.573),66465,53484
472803,2331345218,-62.198185,-58.959160,POINT (-62.198 -58.959),69995,58099
496466,2430830208,-54.500000,158.883333,POINT (-54.500 158.883),66299,162663
496467,2430792906,-54.533333,158.850000,POINT (-54.533 158.850),66315,162648
...,...,...,...,...,...,...
13824320,2237502025,-54.796239,-68.316639,POINT (-54.796 -68.317),66442,53608
13851343,3313344435,-54.843219,-68.554011,POINT (-54.843 -68.554),66464,53494
13854001,3112559449,-54.845658,-68.550094,POINT (-54.846 -68.550),66465,53495
